# Model management
## Saver
After training model, if we want to reuse model, we need to save model parameters. And if we want to continue training,

defining checkpoint would be really helpful. Checkpoint make us be able to start from last checkpoint.

Let's get into it.

##### In TensorFlow

In, **setup phase**, all we need to do is, after completing nodes at last part of setup, add saver node and

in, **run phase**, if you want to save model call save() method by delivering session and checkpoint path!

##### Example is bigcontest data
### Data load

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import os

train_activity_orig = pd.read_csv('C:/Bigcontest/train_activity.csv')
train_label_orig = pd.read_csv('C:/Bigcontest/train_label.csv')

train_activity_mean_orig = train_activity_orig.groupby(['acc_id'], as_index=False).mean()

train_activity_label_orig = pd.merge(train_activity_mean_orig, train_label_orig, how='outer', on='acc_id')

from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=10000, random_state=42)

for train_index, test_index in split.split(train_activity_label_orig, train_activity_label_orig['label']):
    strat_train_dev_set_orig = train_activity_label_orig.loc[train_index]
    strat_test_set_orig = train_activity_label_orig.loc[test_index]

strat_train_dev_set_orig = strat_train_dev_set_orig.reset_index(drop=True)

split2 = StratifiedShuffleSplit(n_splits=1, test_size=20000)

for train_index, test_index in split2.split(strat_train_dev_set_orig, strat_train_dev_set_orig['label']):
    strat_train_set_orig = strat_train_dev_set_orig.loc[train_index]
    strat_dev_set_orig = strat_train_dev_set_orig.loc[test_index]

label_mapping = {"week": 0, "month": 1, "2month": 2, "retained": 3}

for dataset in (strat_train_set_orig, strat_dev_set_orig, strat_test_set_orig):
    dataset['label'] = dataset['label'].map(label_mapping)

X_train_orig = strat_train_set_orig.drop(['acc_id', 'label'], axis=1)
X_dev_orig = strat_dev_set_orig.drop(['acc_id', 'label'], axis=1)
X_test_orig = strat_test_set_orig.drop(['acc_id', 'label'], axis=1)

Y_train_orig = np.array(strat_train_set_orig['label']).reshape(-1,1)
Y_dev_orig = np.array(strat_dev_set_orig['label']).reshape(-1,1)
Y_test_orig = np.array(strat_test_set_orig['label']).reshape(-1,1)

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

Y_train_orig_oh = ohe.fit_transform(Y_train_orig).toarray()
Y_dev_orig_oh = ohe.fit_transform(Y_dev_orig).toarray()
Y_test_orig_oh = ohe.fit_transform(Y_test_orig).toarray()

X_train = X_train_orig.T
X_dev = X_dev_orig.T
X_test = X_test_orig.T

Y_train = Y_train_orig_oh.T
Y_dev = Y_dev_orig_oh.T
Y_test = Y_test_orig_oh.T

X_train = np.array(X_train)
X_dev = np.array(X_dev)
X_test = np.array(X_test)

X_train_orig = np.array(X_train_orig)
X_dev_orig = np.array(X_dev_orig)
X_test_orig = np.array(X_test_orig)

Y_train_orig_oh.shape

## 2. DNN

In [128]:
tf.reset_default_graph() 

n_inputs = 37
n_hidden1 = 300
n_hidden2 = 200
n_hidden3 = 100
n_hidden4 = 50

n_outputs = 4

### 2.1 placeholder

In [129]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
Y = tf.placeholder(tf.int64, shape=(None), name="Y")

### 2.2 layer

In [130]:
from functools import partial

In [131]:
with tf.name_scope("dnn"):
    
    
    
    training = tf.placeholder_with_default(True, shape=(), name="training")
    my_batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)
    

    
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
    bn1 = my_batch_norm_layer(hidden1)
    bn1_act = tf.nn.elu(bn1)
    
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
    bn2 = my_batch_norm_layer(hidden2)
    bn2_act = tf.nn.elu(bn2)
    
    hidden3 = tf.layers.dense(bn2_act, n_hidden3, name="hidden3")
    bn3 = my_batch_norm_layer(hidden3)
    bn3_act = tf.nn.elu(bn3)
    
    hidden4 = tf.layers.dense(bn3_act, n_hidden4, name="hidden4")
    bn4 = my_batch_norm_layer(hidden4)
    bn4_act = tf.nn.elu(bn4)
    
    logits_before_bn = tf.layers.dense(bn4_act, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

### 2.3 compute loss

In [132]:
with tf.name_scope("loss"):
    
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

### 2.4 optimizer

In [133]:
learning_rate = 0.01

with tf.name_scope("train"):
    
    optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum=0.9)
    training_op = optimizer.minimize(loss)

### 2.5 evaluation

tf.argmax(y_, 1)

In [134]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, tf.argmax(Y,1), 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

### 2.6 batch

In [135]:
def random_batch(X_train, Y_train, batch_size):
    
    rnd_indices = np.random.randint(0, len(X_train_orig), batch_size)
    X_batch = X_train[rnd_indices, :]
    Y_batch = Y_train[rnd_indices, :]
    
    return X_batch, Y_batch

# Saver!

In [136]:
saver = tf.train.Saver()

# Create log directory!

In [137]:
from datetime import datetime

def log_dir(prefix=""):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    root_logdir = "tf_logs"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

logdir = log_dir("savetest")

In [138]:
summmary = tf.summary.scalar('LOSS', loss)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

### 2.6 run

In [139]:
init = tf.global_variables_initializer()


n_epochs = 30
batch_size = 32
m = 70000

In [140]:
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

checkpoint_path = "/tmp/save_test.ckpt"
checkpoint_epoch_path = checkpoint_path + ".epoch"
final_model_path = "./final_save_test"

with tf.Session() as sess:
    
    if os.path.isfile(checkpoint_epoch_path):
        with open(checkpoint_epoch_path, "rb") as f:
            start_epoch = int(f.read())
        print("Paused training. Continue epoch.", start_epoch)
        saver.restore(sess, checkpoint_path)
    else:
        start_epoch = 0
        sess.run(init)
        seed = 3
    
    for epoch in range(start_epoch, n_epochs):
        
        num_minibatches = int(m / batch_size)
        epoch_cost = 0
        seed = seed +1
            
        for iteration in range(int(np.ceil(m / batch_size))):
            X_batch, Y_batch = random_batch(X_train_orig, Y_train_orig_oh, batch_size)
            _, minibatch_cost,_ = sess.run([training_op, loss, extra_update_ops], feed_dict={training: True, X: X_batch, Y: Y_batch})
            epoch_cost += minibatch_cost / num_minibatches
            
            
        if epoch % 5 == 0:
            
            print("Cost after epoch %i: %f" %(epoch, epoch_cost))
           
            acc_train = accuracy.eval(feed_dict={X: X_train_orig, Y: Y_train_orig_oh})
            acc_dev = accuracy.eval(feed_dict={X: X_dev_orig, Y: Y_dev_orig_oh})
            print("Train accuracy:", acc_train, "\tDev accuracy", acc_dev)
            
            saver.save(sess, checkpoint_path)
            with open(checkpoint_epoch_path, "wb") as f:
                f.write(b"%d" % (epoch + 1))
        
    saver.save(sess, final_model_path) # Final checkpoint after completing    
    os.remove(checkpoint_epoch_path)    
    
            
            
            
        

Cost after epoch 0: 0.932628
Train accuracy: 0.61887145 	Dev accuracy 0.61555
Cost after epoch 5: 0.848575
Train accuracy: 0.6431 	Dev accuracy 0.63655
Cost after epoch 10: 0.821493
Train accuracy: 0.63388574 	Dev accuracy 0.63105
Cost after epoch 15: 0.806519
Train accuracy: 0.65742856 	Dev accuracy 0.6564
Cost after epoch 20: 0.800604
Train accuracy: 0.6579143 	Dev accuracy 0.65455
Cost after epoch 25: 0.793690
Train accuracy: 0.65422857 	Dev accuracy 0.64945


# Look!

carefully at those saver marks on the code above. 

## Reuse graph!

Save method basically save its structure of graph in .meta file.

So, by tf.train.import_meta_graph(), we can reuse graph

In [144]:
saver = tf.train.import_meta_graph(final_model_path+".meta")

with tf.Session() as sess:
    saver.restore(sess, final_model_path)
    [...]

INFO:tensorflow:Restoring parameters from ./final_save_test
